#### In this file we will plot the results of the grid plot which searches for the optimal spot to aim over different standard deviations

In [1]:
import cv2
import display
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Function to normalize values between 0 and 1
def normalize_exp_val(exp_val, min_val, max_val):
    return (exp_val - min_val) / (max_val - min_val)

# Function to convert normalized values to a color on the red-blue scale
def exp_val_to_color(norm_exp_val):
    red = (1 - norm_exp_val) * 255  # Red decreases as exp_val increases
    blue = norm_exp_val * 255  # Blue increases as exp_val decreases
    return (int(blue), 0, int(red))  # OpenCV uses BGR format

# Load dartboard image
img = display.get_dartboard_image()

# Load data and sort by sd's
max_spots = np.genfromtxt('max_exp_val_over_sds.csv')
max_spots.sort(0)

# Get min and max exp_val to normalize
exp_vals = max_spots[:, 3]
min_exp_val = np.min(exp_vals)
max_exp_val = np.max(exp_vals)

# Iterate through the data and draw circles with normalized color
for sd_data in max_spots:
    exp_val = sd_data[3]
    x, y = display.mm_to_pix((sd_data[1], sd_data[2]))
    sd = sd_data[0]

    # Normalize expected value to map it to the red-blue color scale
    norm_exp_val = normalize_exp_val(exp_val, min_exp_val, max_exp_val)

    # Get the corresponding color
    circle_color = exp_val_to_color(norm_exp_val)

    # Draw the border first (small black border around the circle)
    # Draw the main circle
    img = cv2.circle(img, (x, y), 24, (0, 0, 0), -1)  # Black border
    img = cv2.circle(img, (x, y), 20, circle_color, -1)  # Main circle with red-blue scale

    # Add the standard deviation text inside the circle (make text white)
    font = cv2.FONT_HERSHEY_SIMPLEX
    text = f"SD: {int(sd)}"

    # Adjust font size for better visibility
    font_scale = 0.4  # Slightly larger font scale
    text_size = cv2.getTextSize(text, font, font_scale, 1)[0]
    text_x = x - text_size[0] // 2
    text_y = y + text_size[1] // 2

    # Draw the white text inside the circle
    img = cv2.putText(img, text, (text_x, text_y), font, font_scale, (255, 255, 255), 2, cv2.LINE_AA)

print(img.dtype, img.shape)
success = cv2.imwrite('opt_50000.png', img)
if not success:
    print("Failed to write image file!")

# img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# plt.imshow(img_rgb)
# plt.axis('off')
# plt.show()

uint8 (2000, 2000, 3)
